In [1]:
import pandas as pd
import sqlite3
import json
from tqdm.auto import tqdm

In [2]:
db = sqlite3.connect("datasets.db")
cur = db.cursor()

In [3]:
keys = [
    "id",
    "collapse_field",

    "title",
    
    "rubric_id",
    "rubric_name",
    "rubric_parentId",

    "serial_id",
    "serial_name",
    
    "language_id",
    "language_name",

    "author_id",
    "author_dates",
    "author_fullName",
    
    "available",

    "year_value",
]

In [4]:
! wc -l ../data/books_full.jsonlines

2181593 ../data/books_full.jsonlines


In [ ]:
data = []

k = 0
for line in tqdm(open("../data/books_full.jsonlines"), total=2181593):
    row = json.loads(line)
    res = []
    for key in keys:
        res.append(row.get(key))
    data.append(res)
    k += 1
#     if k > 1000:
#         break

  0%|          | 0/2181593 [00:00<?, ?it/s]

In [6]:
len(data)

2181592

In [7]:
df = pd.DataFrame(data, columns=keys)

In [8]:
df.head(2)

,id,collapse_field,title,rubric_id,rubric_name,rubric_parentId,serial_id,serial_name,language_id,language_name,author_id,author_dates,author_fullName,available,year_value
0,397272,9123ba840c7a63da5c1f13af9f4db75e,Великие правители,13.0,История в целом. Всемирная история,11.0,NaN,None,NaN,None,NaN,None,None,1,2005.0
1,397417,a19bac4ab9cc1df78b95a0534c649a4f,Волею императрицы : Ист.романы,479.0,Художественная литература,NaN,4495.0,Государи Руси Великой,1.0,Русский,126874.0,None,Щепкина А. В.,1,1996.0


In [9]:
df["collapse_field"].nunique()

1909342

In [10]:
def toint(x):
    try:
        if type(x) == int:
            return x
        if type(x) == str and "." in x:
            return int(x.split(".")[0])
        return int(x)
    except:
#         print(x)
        return -1

In [11]:
for key in ["rubric_id", "rubric_parentId", "serial_id", "language_id", "author_id", "year_value"]:
    df[key] = df[key].apply(toint)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181592 entries, 0 to 2181591
Data columns (total 15 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   id               int64 
 1   collapse_field   object
 2   title            object
 3   rubric_id        int64 
 4   rubric_name      object
 5   rubric_parentId  int64 
 6   serial_id        int64 
 7   serial_name      object
 8   language_id      int64 
 9   language_name    object
 10  author_id        int64 
 11  author_dates     object
 12  author_fullName  object
 13  available        int64 
 14  year_value       int64 
dtypes: int64(8), object(7)
memory usage: 249.7+ MB


In [15]:
df.to_sql("books", con=db)

In [16]:
for key in ["rubric_id", "rubric_parentId", "serial_id", "language_id", "author_id", "year_value"]:
    cur.execute(f"UPDATE books SET {key} = NULL WHERE {key} = -1")
    db.commit()

In [17]:
PATH = "/home/dkbrz/github/moscow_books/raw_data/dataset/datasets_biblioteki/datasets_2/"

In [18]:
data = []

In [19]:
states = {
    "Размещен в фонде": 0,
    "Переоформлен": 1,
    "На руках": 2,
    "Принят у читателя": 3,
    "Возвращен на пункт книговыдачи": 4,
    "Прибыл": 5,
    "Отправлен": 6,
}

In [21]:
data = []
for i in tqdm(range(1, 17)):
    df = pd.read_csv(PATH + f"circulaton_{i}.csv", encoding="cp1251", sep=";")
    df = df[["catalogueRecordID", "readerID", "state", "startDate", "finishDate"]]
    df["startDate"] = df["startDate"].apply(lambda x: "-".join(reversed(x.split("."))))
    df["finishDate"] = df["finishDate"].apply(lambda x: "-".join(reversed(x.split("."))))
    df["state"] = df["state"].apply(lambda x: states[x])
    df = df[df["state"] > 1]
    data.append(df)

  0%|          | 0/16 [00:00<?, ?it/s]

In [22]:
df = pd.concat(data)

In [23]:
df.shape

(605486, 5)

In [24]:
df.sort_values(by="startDate").to_sql("circulation_short", con=db, index=False)

In [26]:
df.state.value_counts()

2    580305
3     22472
6      1679
4       990
5        40
Name: state, dtype: int64